# Extract and align data from Onix, Harp, Sleap, and photometry
## Cohort 1 and 2 working, Cohort 0: onix_digital Clock column is 0, explore why and/or use timestamps instead 

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp
import plotly.express as px

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
cohort0 = False #only read harp data when it exists, not in Cohort0 
cohort2 = False

#Cohort 1 vestibular mismatch, multiple OnixDigital files 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 1 with clock accumulation issue marked on google sheet, seems fine though
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2719-2024-12-12T13-59-38') #multiple onix_digital file

#Cohort 1 visual mismatch 
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2718-2024-12-10T12-57-02') 

#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True

#Cohort 2 N.B. no videodata in this test set 
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_like_test_data/2025-01-13T15-47-26')

#Cohort 2 longer test YES OnixHarp! 
#N.B. no photometry in this test set (neitjer videos, but yes video_data)
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test_longer/2025-02-10T08-18-59')
 
photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"

h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
# h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
# h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)

session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
#photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
%%time
#read metadata in 2 different ways (to df or to dict, to decide which one is better in the future)
print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
#session_settings = utils.read_SessionSettings(data_path) #Hilde's, creates prety dict, not aware of multiple files

# read experiment events, video, processed photometry 
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)

print ("Loading processed fluorescence")
photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
print ("Loading processed fluorescence info")
photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
print ("Loading processed fluorescence events")
photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')

if not cohort2:
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data 
print ("Loading OnixDigital")
onix_digital = utils.load_2(onix_digital_reader, data_path)
print ("Loading OnixAnalogFrameClock")
onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData")
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True) #channels is a list of AI lines, 0-11

#read harp streams and separate registers if needed 
print ("Loading H1 and H2 streams as dict or df")
harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict 

#read syncronising signal between HARP and ONIX
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    # removing possible outliers 
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )
    onix_harp["HarpTime"] = onix_harp["HarpTime"] + 1 # known issue with current version of ONIX, harp timestamps lag 1 second
    print ("Warning: HarpTime +1s to account for know issue with ONIX")

# print (" ")
# print ("loading separate registers from H1 and H2 data")
print ("Loading camera triggers")
camera_triggers = utils.load_harp(h1_reader.Cam0Event, h1_datafolder) #assumes Cam0 triggers both cameras
print ("Loading flow sensor data")
flow_sensor = utils.load_harp(h1_reader.OpticalTrackingRead, h1_datafolder)
print ("Done Loading")

In [ ]:
import importlib
importlib.reload(utils)
importlib.reload(process) # Forces Python to reload the updated module
None

In [ ]:
%%time
(
    conversions, 
    photometry_sync_events, 
    harp_to_onix_clock, 
    onix_time_to_photometry, 
    onix_to_harp_timestamp,
    photometry_to_harp_time
) = process.photometry_harp_onix_synchronisation(
    onix_analog_data=onix_analog_data,
    onix_analog_clock=onix_analog_clock,
    onix_analog_framecount=onix_analog_framecount,
    onix_digital=onix_digital,
    onix_harp=onix_harp,
    photometry_events=photometry_events,
    verbose=True
)


In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# User-defined time window (in seconds)
window_start = -1  # Start time relative to halt event
window_stop = 5    # Stop time relative to halt event

how_many_to_plot = len(experiment_events.query("Event == 'Apply halt: 2s'"))  # Use all halts
#how_many_to_plot = 2

print ("Found",len(experiment_events.query("Event == 'Apply halt: 2s'"))," number of halts, plotting ", how_many_to_plot)

# Efficient filtering
block_halts = experiment_events.query("Event == 'Apply halt: 2s'").iloc[:how_many_to_plot]

# Define colors for each signal type
flow_x_color = "blue"
flow_y_color = "orange"
photodiode_color = "purple"
dfF_470_color = "green"
dfF_560_color = "red"

# Initialize lists for storing aligned data
aligned_time = np.linspace(window_start, window_stop, 500)  # Fixed time grid for averaging
flow_x_aligned = []
flow_y_aligned = []
photodiode_aligned = []
dfF_470_aligned = []
dfF_560_aligned = []

plt.figure(figsize=(10, 6))
fig, ax1 = plt.subplots(figsize=(10, 6))

for idx, halt_time in enumerate(block_halts.index):
    halt_time_seconds = halt_time.timestamp()

    # Define new time range based on user input
    min_time = halt_time + pd.DateOffset(seconds=window_start)
    max_time = halt_time + pd.DateOffset(seconds=window_stop)

    # Extract Optical Tracking Data
    optical_x = harp_streams['OpticalTrackingRead0X(46)'].loc[min_time:max_time].dropna()
    optical_y = harp_streams['OpticalTrackingRead0Y(46)'].loc[min_time:max_time].dropna()

    if not optical_x.empty and not optical_y.empty:
        optical_x_rel = (optical_x.index.astype("int64") / 1e9) - halt_time_seconds
        optical_y_rel = (optical_y.index.astype("int64") / 1e9) - halt_time_seconds

        ax1.plot(optical_x_rel, optical_x, label='Flow X', color=flow_x_color, alpha=0.5)
        ax1.plot(optical_y_rel, optical_y, label='Flow Y', color=flow_y_color, alpha=0.5)

        # Interpolate for averaging
        flow_x_interp = np.interp(aligned_time, optical_x_rel, optical_x, left=np.nan, right=np.nan)
        flow_y_interp = np.interp(aligned_time, optical_y_rel, optical_y, left=np.nan, right=np.nan)
        flow_x_aligned.append(flow_x_interp)
        flow_y_aligned.append(flow_y_interp)

# Formatting for Optical Flow
ax1.set_xlabel("Relative Time (s)")
ax1.set_ylabel("Tracking Readout")
ax1.set_title("Optical Flow, Photodiode, and Photometry")

# Create separate y-axis for Photodiode
ax3 = ax1.twinx()
ax3.spines.right.set_position(("outward", 60))
ax3.set_ylabel("Photodiode Signal")
ax3.set_ylim([0, 1.2])

# Process ONIX signals (Photodiode)
for idx, halt_time in enumerate(block_halts.index):
    halt_time_seconds = halt_time.timestamp()

    onix_sec_start_time = harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_start)
    onix_sec_stop_time = harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_stop)

    onix_sec_start_index = np.searchsorted(onix_analog_clock, onix_sec_start_time)
    onix_sec_stop_index = np.searchsorted(onix_analog_clock, onix_sec_stop_time)

    # Convert Onix timestamps to relative seconds
    onix_time_rel = (onix_to_harp_timestamp(onix_analog_clock[onix_sec_start_index:onix_sec_stop_index])
                     .astype("int64") / 1e9) - halt_time_seconds

    ax3.plot(
        onix_time_rel,
        onix_analog_data[onix_sec_start_index:onix_sec_stop_index],
        label="Photodiode",
        color=photodiode_color, alpha=0.8
    )

    # Interpolate for averaging
    photodiode_interp = np.interp(aligned_time, onix_time_rel, onix_analog_data[onix_sec_start_index:onix_sec_stop_index],
                                  left=np.nan, right=np.nan)
    photodiode_aligned.append(photodiode_interp)

# Create second y-axis for photometry
ax2 = ax1.twinx()
ax2.set_ylabel("Fluorescence Signal")

# Convert `TimeStamp` column to seconds if not already done
if "TimeStamp" in photometry_data.columns:
    photometry_data = photometry_data.set_index("TimeStamp")

for idx, halt_time in enumerate(block_halts.index):
    halt_time_seconds = halt_time.timestamp()

    photometry_sec_start_time = onix_time_to_photometry(harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_start))
    photometry_sec_stop_time = onix_time_to_photometry(harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] + window_stop))

    # Select photometry data using proper time range
    photometry_sec = photometry_data.loc[photometry_sec_start_time:photometry_sec_stop_time]

    if not photometry_sec.empty:
        # Convert photometry timestamps to relative seconds
        photometry_time_rel = (photometry_to_harp_time(photometry_sec.index).astype("int64") / 1e9) - halt_time_seconds

        ax2.plot(
            photometry_time_rel,
            photometry_sec['dfF_560'],
            label='dfF_560',
            color=dfF_560_color
        )

        ax2.plot(
            photometry_time_rel,
            photometry_sec['dfF_470'],
            label='dfF_470',
            color=dfF_470_color
        )

        # Interpolate for averaging
        dfF_560_interp = np.interp(aligned_time, photometry_time_rel, photometry_sec['dfF_560'], left=np.nan, right=np.nan)
        dfF_470_interp = np.interp(aligned_time, photometry_time_rel, photometry_sec['dfF_470'], left=np.nan, right=np.nan)
        dfF_560_aligned.append(dfF_560_interp)
        dfF_470_aligned.append(dfF_470_interp)

plt.show()

# **Second plot: Average and Error Shading**
plt.figure(figsize=(10, 6))
fig, ax1 = plt.subplots(figsize=(10, 6))

# Compute mean and std across trials
flow_x_mean = np.nanmean(flow_x_aligned, axis=0)
flow_y_mean = np.nanmean(flow_y_aligned, axis=0)
photodiode_mean = np.nanmean(photodiode_aligned, axis=0)
dfF_560_mean = np.nanmean(dfF_560_aligned, axis=0)
dfF_470_mean = np.nanmean(dfF_470_aligned, axis=0)

flow_x_std = np.nanstd(flow_x_aligned, axis=0)
flow_y_std = np.nanstd(flow_y_aligned, axis=0)
photodiode_std = np.nanstd(photodiode_aligned, axis=0)
dfF_560_std = np.nanstd(dfF_560_aligned, axis=0)
dfF_470_std = np.nanstd(dfF_470_aligned, axis=0)

# Plot means with shaded error regions
ax1.plot(aligned_time, flow_x_mean, label="Flow X (Mean)", color=flow_x_color)
ax1.fill_between(aligned_time, flow_x_mean - flow_x_std, flow_x_mean + flow_x_std, color=flow_x_color, alpha=0.3)

ax1.plot(aligned_time, flow_y_mean, label="Flow Y (Mean)", color=flow_y_color)
ax1.fill_between(aligned_time, flow_y_mean - flow_y_std, flow_y_mean + flow_y_std, color=flow_y_color, alpha=0.3)

ax3 = ax1.twinx()
ax3.plot(aligned_time, photodiode_mean, label="Photodiode (Mean)", color=photodiode_color)
ax3.fill_between(aligned_time, photodiode_mean - photodiode_std, photodiode_mean + photodiode_std, color=photodiode_color, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(aligned_time, dfF_560_mean, label="dfF_560 (Mean)", color=dfF_560_color)
ax2.fill_between(aligned_time, dfF_560_mean - dfF_560_std, dfF_560_mean + dfF_560_std, color=dfF_560_color, alpha=0.3)

ax2.plot(aligned_time, dfF_470_mean, label="dfF_470 (Mean)", color=dfF_470_color)
ax2.fill_between(aligned_time, dfF_470_mean - dfF_470_std, dfF_470_mean + dfF_470_std, color=dfF_470_color, alpha=0.3)

plt.show()
